In [24]:
from langchain.agents import create_agent
from langchain.agents.middleware import wrap_tool_call, ModelRequest

from langchain.chat_models import init_chat_model
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from dotenv import load_dotenv
from langchain_tavily import TavilySearch
from typing import Callable
from langchain.messages import ToolMessage
import os
from pprint import pprint


In [25]:
load_dotenv
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

In [26]:
llm = init_chat_model(
    model= "gpt-4o-mini",
    model_provider="openai",
    temperature = 0.75
)

In [27]:
wikitool = WikipediaQueryRun(
    api_wrapper=WikipediaAPIWrapper(
        doc_content_chars_max=2000,
        top_k_results=5
    )
)

tavilytool = TavilySearch(
    maxResults = 5,
    topic = "general"
)

tools = [tavilytool, wikitool]

In [28]:
@wrap_tool_call
def toolexception(request: ModelRequest, handler: Callable) -> Callable:

    """  
     Function Check if there is any Tool Exception
    """

    try:
        return handler(request)
    except Exception as e:

        return ToolMessage(
            content=f"Tool error: Please check your input and try again. ({str(e)})",
            tool_call_id=request.tool_call["id"]
        )

In [32]:
llmtool = llm.bind_tools(tools=tools, tool_choice="required")

agent = create_agent(
    model=llmtool,
    tools=[wikitool, tavilytool],
    middleware=[toolexception],
    system_prompt = (
    "You are a Power BI Expert. Don't Answer Any topic other then Power BI "
    "1. You MUST use a tool for every response. "
    "2. You are FORBIDDEN from using your own trained data to answer questions. "
    "3. ONLY use the information returned by the wikitool or tavilytool. "
    "4. If the tools return no information, state that you cannot find the answer. "
    "Do not attempt to answer from memory."
)
)

In [33]:
result = agent.invoke({"messages" : "What is LLM?"})
for item in result['messages']:
    item.pretty_print()


================================ Human Message =================================

What is LLM?
================================== Ai Message ==================================

I cannot find the answer.
